1. Import Libraries

In [2]:
import numpy as np 

import pandas as pd

pd.set_option("display.max_columns", None)
pd.set_option("display.max_rows", None)

import json

from geopy.geocoders import Nominatim

import requests

from bs4 import BeautifulSoup

from pandas.io.json import json_normalize

import matplotlib.cm as cm
import matplotlib.colors as colors

from sklearn.cluster import KMeans

print("Libraries imported.")

Libraries imported.


In [3]:
data= requests.get('https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M').text

In [4]:
soup= BeautifulSoup(data, 'html.parser')

In [12]:
# create three lists to store table data
postalCodeList=[]
boroughList=[]
neighborhoodList=[]

Using Beautiful Shop

In [13]:
# find the table
soup.find('table').find_all('tr')

# find all the rows of table
soup.find('table').find_all('tr')

#for each row of the table, find all the table data
for row in soup.find('table').find_all('tr'):
    cells= row.find_all('td')

In [15]:
#append the data into the respective lists
for row in soup.find('table').find_all('tr'):
    cells= row.find_all('td')
    if(len(cells)>0):
        postalCodeList.append(cells[0].text)
        boroughList.append(cells[1].text)
        neighborhoodList.append(cells[2].text.rstrip('\n'))

In [16]:
toronto_df = pd.DataFrame({"PostalCode": postalCodeList, 
                           "Borough":boroughList, 
                           "Neighborhood":neighborhoodList})

toronto_df.head

<bound method NDFrame.head of     PostalCode           Borough  \
0          M1A      Not assigned   
1          M2A      Not assigned   
2          M3A        North York   
3          M4A        North York   
4          M5A  Downtown Toronto   
5          M6A        North York   
6          M6A        North York   
7          M7A      Queen's Park   
8          M8A      Not assigned   
9          M9A      Queen's Park   
10         M1B       Scarborough   
11         M1B       Scarborough   
12         M2B      Not assigned   
13         M3B        North York   
14         M4B         East York   
15         M4B         East York   
16         M5B  Downtown Toronto   
17         M5B  Downtown Toronto   
18         M6B        North York   
19         M7B      Not assigned   
20         M8B      Not assigned   
21         M9B         Etobicoke   
22         M9B         Etobicoke   
23         M9B         Etobicoke   
24         M9B         Etobicoke   
25         M9B         Etobicoke  

In [17]:
#drop cells with Borough not assigned
toronto_df_dropna= toronto_df[toronto_df.Borough !="Not assigned"].reset_index(drop=True)
toronto_df_dropna.head()

,PostalCode,Borough,Neighborhood
0,M3A,North York,Parkwoods
1,M4A,North York,Victoria Village
2,M5A,Downtown Toronto,Harbourfront
3,M6A,North York,Lawrence Heights
4,M6A,North York,Lawrence Manor


In [19]:
#group neighborhoods in the same borough
toronto_df_grouped= toronto_df_dropna.groupby(["PostalCode", "Borough"], as_index=False).agg(lambda x: ",".join(x))
toronto_df_grouped.head()

,PostalCode,Borough,Neighborhood
0,M1B,Scarborough,"Rouge,Malvern,Rouge,Malvern"
1,M1C,Scarborough,"Highland Creek,Rouge Hill,Port Union,Highland ..."
2,M1E,Scarborough,"Guildwood,Morningside,West Hill,Guildwood,Morn..."
3,M1G,Scarborough,"Woburn,Woburn"
4,M1H,Scarborough,"Cedarbrae,Cedarbrae"


In [22]:
#create new test frame
column_names=["PostalCode", "Borough", "Neighborhood"] 
test_df= pd.DataFrame(columns=column_names)

test_list= ["M5G","M2H","M4B", "M1J", "M4G", "M4M","M1R","M9V","M9L","M5V", "M1B", "M5A"]

for postcode in test_list:
    test_df= test_df.append(toronto_df_grouped[toronto_df_grouped["PostalCode"]==postcode], ignore_index=True)

test_df

,PostalCode,Borough,Neighborhood
0,M5G,Downtown Toronto,"Central Bay Street,Central Bay Street"
1,M2H,North York,"Hillcrest Village,Hillcrest Village"
2,M4B,East York,"Woodbine Gardens,Parkview Hill,Woodbine Garden..."
3,M1J,Scarborough,"Scarborough Village,Scarborough Village"
4,M4G,East York,"Leaside,Leaside"
5,M4M,East Toronto,"Studio District,Studio District"
6,M1R,Scarborough,"Maryvale,Wexford,Maryvale,Wexford"
7,M9V,Etobicoke,"Albion Gardens,Beaumond Heights,Humbergate,Jam..."
8,M9L,North York,"Humber Summit,Humber Summit"
9,M5V,Downtown Toronto,"CN Tower,Bathurst Quay,Island airport,Harbourf..."


In [23]:
#print the number of rows of the cleaned dataframe
toronto_df_grouped.shape

(103, 3)